#Flights Notebook

In [23]:
import requests
import json
import pandas as pd


In [24]:
#Coordinates for United States
longitudeMin, latitudeMin =- 125.974, 30.038
longitudeMax, latitudeMax =- 68.748, 52.214

In [25]:
# API QUERY
userName = ''
password = ''

urlData = 'https://' + userName + ':' + password + '@opensky-network.org/api/states/all?' + 'lamin=' + str(latitudeMin) + '&lomin=' + str(longitudeMin) + '&lamax=' + str(latitudeMax) + '&lomax=' + str(longitudeMax)
response = requests.get(urlData).json()

In [26]:
# Load data to Pandas
colName = ['icao24', 'callsign', 'origin_country', 'time_position', 'last-contact', 'long', 'lat', 'baro_altitude', 'on_ground', 'velocity', 
'true_track', 'vertical_rate', 'sensors', 'geo_latitude', 'squawk', 'spi', 'position_source']

flightDF = pd.DataFrame(response['states'], columns = colName)
flightDF = flightDF.fillna('No Data')
flightDF.head()

,icao24,callsign,origin_country,time_position,last-contact,long,lat,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_latitude,squawk,spi,position_source
0,a665ae,IFL511,United States,1671320741,1671320741,-87.8479,35.9731,10972.8,False,154.10,260.00,0.65,No Data,10858.5,No Data,False,0
1,aa8c39,UAL1801,United States,1671320742,1671320742,-120.1364,37.7031,10972.8,False,251.93,259.17,0.0,No Data,11003.28,2734,False,0
2,ad4f1d,N957ES,United States,1671320687,1671320741,-97.7511,35.4054,815.34,False,52.90,130.66,-1.63,No Data,838.2,0417,False,0
3,ade18c,AAL2360,United States,1671320742,1671320742,-100.8222,33.9486,9144.0,False,204.65,288.62,0.0,No Data,9250.68,No Data,False,0
4,abd9d9,SWA549,United States,1671320737,1671320737,-95.7211,38.6814,10972.8,False,183.53,260.64,0.0,No Data,10751.82,No Data,False,0


In [27]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, STAMEN_TERRAIN
from bokeh.models import HoverTool, LabelSet, ColumnDataSource
import numpy as np

In [28]:
#FUNCTION TO CONVERT GCS WGS84 TO WEB MERCATOR POINT
def wgs84WebMercatorPoint(lon,lat):
    k = 6378137
    x= lon * (k * np.pi/180.0)
    y= np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x,y

#DATA FRAME
def wgs84ToWebMercator(df, lon="long", lat="lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

 #COORDINATE CONVERSION
xy_min = wgs84WebMercatorPoint(longitudeMin,latitudeMin)
xy_max = wgs84WebMercatorPoint(longitudeMax,latitudeMax)
wgs84ToWebMercator(flightDF)
flightDF['rot_angle'] = flightDF['true_track'] *- 1 #Rotation angle
icon_url = 'https://.....' #Icon url
flightDF['url'] = icon_url

#FIGURE SETTING
x_range,y_range = ([xy_min[0], xy_max[0]], [xy_min[1], xy_max[1]])
p = figure(x_range = x_range, y_range = y_range, x_axis_type = 'mercator', y_axis_type = 'mercator', sizing_mode = 'scale_width', plot_height = 300)


#PLOT BASEMAP AND AIRPLANE POINTS
flightSource = ColumnDataSource(flightDF)
tile_prov = get_provider(STAMEN_TERRAIN)
p.add_tile(tile_prov,level = 'image')
p.image_url(url = 'url', x = 'x', y = 'y', source = flightSource, anchor = 'center', angle_units = 'deg', angle = 'rot_angle', h_units = 'screen', w_units = 'screen', w = 40, h = 40)
p.circle('x','y', source = flightSource, fill_color = 'red', hover_color = 'yellow', size = 10, fill_alpha = 0.8, line_width = 0)

#HOVER INFORMATION AND LABEL
my_hover = HoverTool()
my_hover.tooltips=[('Call sign','@callsign'),('Origin Country','@origin_country'),('velocity(m/s)','@velocity'),('Altitude(m)','@baro_altitude')]
labels = LabelSet(x='x', y='y', text='callsign', level='glyph',
            x_offset=5, y_offset=5, source=flightSource, render_mode='canvas',background_fill_color='white',text_font_size="8pt")
p.add_tools(my_hover)
p.add_layout(labels)

show(p)
